In [2]:
import os

import numpy as np
from scipy.sparse import coo_matrix
from sklearn.preprocessing import normalize
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

In [4]:
class PairData(Data):
    def __init__(self, edge_index_s=None, x_s=None, edge_attr_s=None, edge_list_s=None,
                       edge_index_t=None, x_t=None, edge_attr_t=None, edge_list_t=None,
                       y=None):
        super().__init__()
        self.edge_index_s = edge_index_s
        self.x_s = x_s
        self.edge_attr_s = edge_attr_s
        self.edge_list_s = edge_list_s
        self.edge_index_t = edge_index_t
        self.x_t = x_t
        self.edge_attr_t = edge_attr_t
        self.edge_list_t = edge_list_t
        self.y = y

    def __inc__(self, key, value, *args, **kwargs):
        if key == 'edge_index_s':
            return self.x_s.size(0)
        if key == 'edge_index_t':
            return self.x_t.size(0)
        if key == 'edge_list_s':
            return self.edge_attr_s.size(0)
        if key == 'edge_list_t':
            return self.edge_attr_t.size(0)
        if key == 'y':
            return self.y.size(0)
        else:
            return super().__inc__(key, value, *args, **kwargs)

In [5]:
x1 = torch.tensor([
    [1, 2],
    [3, 4],
    [5, 6],
    [7, 8]
])
edge_idx1 = torch.tensor([
    [0, 0, 0],
    [1, 2, 3]
])
edge_attr1 = torch.tensor([
    [-1, -2, -3],
    [-4, -5, -6],
    [-7, -8, -9]
])
# edge_list1 = torch.arange(len(edge_attr1))
edge_list1 = torch.tensor([[1, 2, 0], [2, 1, 0], [0, 1, 2]])

data1 = Data(x=x1, edge_index=edge_idx1, edge_attr=edge_attr1)

In [6]:
x2 = torch.tensor([
    [1, 2],
    [2, 3],
    [3, 4],
    [4, 5],
    [5, 6]
])
edge_idx2 = torch.tensor([
    [0, 0, 0, 0],
    [1, 2, 3, 4]
])
edge_attr2 = torch.tensor([
    [-1, -2, -3],
    [-3, -4, -5],
    [-5, -6, -7],
    [-7, -8, -9]
])
edge_list2 = torch.tensor([[1, 2, 0], [2, 1, 0], [0, 1, 2], [0, 2, 1]])

data2 = Data(x=x2, edge_index=edge_idx2, edge_attr=edge_attr2)

In [7]:
y = torch.tensor([1])

data_p = PairData(edge_index_s=edge_idx1, x_s=x1, edge_attr_s=edge_attr1, edge_list_s=edge_list1,
                  edge_index_t=edge_idx2, x_t=x2, edge_attr_t=edge_attr2, edge_list_t=edge_list2,
                  y=y)

In [8]:
data_list = [data_p, data_p]
loader = DataLoader(data_list, batch_size=2, follow_batch=['x_s', 'x_t', 'y'])

In [9]:
batch = next(iter(loader))

In [10]:
batch

PairDataBatch(edge_index_s=[2, 6], x_s=[8, 2], x_s_batch=[8], x_s_ptr=[3], edge_attr_s=[6, 3], edge_list_s=[6, 3], edge_index_t=[2, 8], x_t=[10, 2], x_t_batch=[10], x_t_ptr=[3], edge_attr_t=[8, 3], edge_list_t=[8, 3], y=[2], y_batch=[2], y_ptr=[3])

In [11]:
batch.x_s_batch

tensor([0, 0, 0, 0, 1, 1, 1, 1])

In [15]:
batch.edge_list_s

tensor([[1, 2, 0],
        [2, 1, 0],
        [0, 1, 2],
        [4, 5, 3],
        [5, 4, 3],
        [3, 4, 5]])

In [12]:
batch.x_s

tensor([[1, 2],
        [3, 4],
        [5, 6],
        [7, 8],
        [1, 2],
        [3, 4],
        [5, 6],
        [7, 8]])

In [13]:
batch.edge_attr_s

tensor([[-1, -2, -3],
        [-4, -5, -6],
        [-7, -8, -9],
        [-1, -2, -3],
        [-4, -5, -6],
        [-7, -8, -9]])

In [14]:
batch.edge_list_s

tensor([[1, 2, 0],
        [2, 1, 0],
        [0, 1, 2],
        [4, 5, 3],
        [5, 4, 3],
        [3, 4, 5]])

In [26]:
batch.edge_index_s

tensor([[0, 0, 0, 4, 4, 4],
        [1, 2, 3, 5, 6, 7]])

In [38]:
torch.full((1, batch.edge_index_s.shape[-1]), 1).squeeze()

tensor([1, 1, 1, 1, 1, 1])

In [42]:
torch.sparse_coo_tensor(indices=batch.edge_index_s,
                        values=torch.full((1, batch.edge_index_s.shape[-1]), 1).squeeze(),
                        size=(batch.x_s.shape[0], batch.x_s.shape[0])).to_dense()

tensor([[0, 1, 1, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0]])

In [13]:
batch.edge_attr

tensor([[-1, -2, -3],
        [-4, -5, -6],
        [-7, -8, -9],
        [-1, -2, -3],
        [-3, -4, -5],
        [-5, -6, -7],
        [-7, -8, -9]])

In [3]:
torch.tensor([[1, 2, 3], [4, 5, 6]]) + 1

tensor([[2, 3, 4],
        [5, 6, 7]])

In [ ]:
0, 1, 0
0, 2, 1
3, 4, 2
3, 5, 3

In [27]:
import multiprocessing
from time import time

In [24]:
start_time = time()
l = []
l2 = list(range(1, 12))

for i in l2:
    l.append(i)

print(l)
end_time = time()

print(end_time - start_time)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
0.00015878677368164062


In [35]:
def dothing(*L2):  # the managed list `L` passed explicitly.
    L = []
    for i in L2:
        L.append(i)
    return L

In [23]:
start_time = time()
with multiprocessing.Manager() as manager:
    L = manager.list()  # <-- can be shared between processes.
    L2 = [[1, 2], [3, 4], [5, 6, 7], [8, 9, 10, 11]]
    processes = []
    for i in range(4):
        p = multiprocessing.Process(target=dothing, args=(L, L2[i]))  # Passing the list
        p.start()
        processes.append(p)
    for p in processes:
        p.join()
    print(L)

end_time = time()

print(end_time - start_time)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
0.04671788215637207


In [39]:
start_time = time()
L = []  # <-- can be shared between processes.
L2 = [[1, 2], [3, 4], [5, 6, 7], [8, 9, 10, 11]]
with multiprocessing.Pool() as pool:
    l_list = pool.starmap(dothing, L2)
print(l_list)

end_time = time()

print(end_time - start_time)

[[1, 2], [3, 4], [5, 6, 7], [8, 9, 10, 11]]
0.03792405128479004


In [2]:
import torch
from torch_geometric.utils import add_self_loops, get_self_loop_attr

edge_index = torch.tensor([[0, 1, 1],
                           [1, 0, 2]])
edge_weight = torch.tensor([[0.2, 0.3, 0.5],
                            [  1,   2,   3],
                            [  0, 0.3, 0.4]])
get_self_loop_attr(edge_index, edge_weight)

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])

In [2]:
add_self_loops(edge_index, edge_weight)

(tensor([[0, 1, 1, 0, 1, 2],
         [1, 0, 2, 0, 1, 2]]),
 tensor([[0.2000, 0.3000, 0.5000],
         [1.0000, 2.0000, 3.0000],
         [0.0000, 0.3000, 0.4000],
         [1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000]]))

In [56]:
a = torch.randn((3, 4))
l = [a, a + 1]
print(a)
b = torch.stack(l, dim=-1)
print(b, b.shape)

tensor([[ 0.1253, -0.1214,  1.3726,  0.2710],
        [ 0.2165, -0.2144, -0.2145, -0.4526],
        [-0.1415,  0.1179,  0.0156, -0.3729]])
tensor([[[ 0.1253,  1.1253],
         [-0.1214,  0.8786],
         [ 1.3726,  2.3726],
         [ 0.2710,  1.2710]],

        [[ 0.2165,  1.2165],
         [-0.2144,  0.7856],
         [-0.2145,  0.7855],
         [-0.4526,  0.5474]],

        [[-0.1415,  0.8585],
         [ 0.1179,  1.1179],
         [ 0.0156,  1.0156],
         [-0.3729,  0.6271]]]) torch.Size([3, 4, 2])


In [55]:
print(b.T, b.T.shape)

tensor([[[-0.1021, -0.7876,  0.3487]],

        [[ 0.8979,  0.2124,  1.3487]]]) torch.Size([2, 1, 3])


In [76]:
c = torch.nn.Linear(2, 1)
d = torch.nn.Linear(4, 4)
d.weight = torch.nn.Parameter(torch.full((4, 4), 1.), requires_grad=False)
print(c)
print(d)
e = c(b).squeeze()
print(e, e.shape)
print(c.weight, c.bias)

print(d.weight, d.bias)
print(d(e))


Linear(in_features=2, out_features=1, bias=True)
Linear(in_features=4, out_features=4, bias=True)
tensor([[-0.8413, -0.8104, -0.9974, -0.8595],
        [-0.8527, -0.7988, -0.7988, -0.7690],
        [-0.8079, -0.8404, -0.8276, -0.7790]], grad_fn=<SqueezeBackward0>) torch.Size([3, 4])
Parameter containing:
tensor([[ 0.5620, -0.6871]], requires_grad=True) Parameter containing:
tensor([-0.1385], requires_grad=True)
Parameter containing:
tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]]) Parameter containing:
tensor([ 0.1455, -0.3635, -0.0089,  0.0744], requires_grad=True)
tensor([[-3.3631, -3.8722, -3.5175, -3.4343],
        [-3.0738, -3.5828, -3.2281, -3.1449],
        [-3.1094, -3.6184, -3.2637, -3.1805]], grad_fn=<AddmmBackward0>)


In [19]:
import torch
l = torch.tensor([[1, 2],
                  [2, 3],
                  [4, 6]], dtype=torch.float)

m = (l - torch.mean(l, dim=0)) / torch.std(l, dim=0)
m

tensor([[-0.8729, -0.8006],
        [-0.2182, -0.3203],
        [ 1.0911,  1.1209]])

In [17]:
torch.mean(l, dim=0)

tensor([2.3333, 3.6667])

In [18]:
torch.std(l, dim=0)

tensor([1.5275, 2.0817])

In [22]:
torch.std(m, dim=0)

tensor([1., 1.])

In [25]:
torch.sum(l, dim=-1)

tensor([ 3.,  5., 10.])

In [31]:
D = torch.diag(torch.tensor([1., 2., 1.]))
D

tensor([[1., 0., 0.],
        [0., 2., 0.],
        [0., 0., 1.]])

In [32]:
D_inv = D.inverse()
D_inv

tensor([[1.0000, 0.0000, -0.0000],
        [0.0000, 0.5000, -0.0000],
        [0.0000, 0.0000, 1.0000]])

In [33]:
D_inv_sqrt = D_inv ** 0.5
D_inv_sqrt

tensor([[1.0000, 0.0000, -0.0000],
        [0.0000, 0.7071, -0.0000],
        [0.0000, 0.0000, 1.0000]])

In [34]:
A = torch.tensor([[0., 1., 0.],
                  [1., 0., 1.],
                  [0., 1., 0.]])
A

tensor([[0., 1., 0.],
        [1., 0., 1.],
        [0., 1., 0.]])

In [36]:
torch.eye(3, dtype=torch.float) - D_inv_sqrt @ A @ D_inv_sqrt

tensor([[ 1.0000, -0.7071,  0.0000],
        [-0.7071,  1.0000, -0.7071],
        [ 0.0000, -0.7071,  1.0000]])

In [37]:
torch.linalg.norm(l, ord='fro')

tensor(8.3666)

In [5]:
import numpy as np

a = np.array([[1, 2, 3], [4, 5, 6]])
a

array([[1, 2, 3],
       [4, 5, 6]])

In [13]:
a.T

array([[1, 4],
       [2, 5],
       [3, 6]])

In [14]:
a.T[[0, 2]]

array([[1, 4],
       [3, 6]])

In [25]:
import torch
import torch.nn.functional as F

a = [[2, 4, 4, 5],
     [1, 2, 3, 9],
     [4, 5, 6, 1]]

adj = [[0, 1, 1],
       [1, 0, 0],
       [0, 1, 0]]

a = torch.tensor(a).double()
adj = torch.tensor(adj).double()

b = adj @ a

print(a)
print(b)

tensor([[2., 4., 4., 5.],
        [1., 2., 3., 9.],
        [4., 5., 6., 1.]], dtype=torch.float64)
tensor([[ 5.,  7.,  9., 10.],
        [ 2.,  4.,  4.,  5.],
        [ 1.,  2.,  3.,  9.]], dtype=torch.float64)


In [26]:
w_key = [
  [0, 0, 1],
  [1, 1, 0],
  [0, 1, 0],
  [1, 1, 0]
]
w_query = [
  [1, 0, 1],
  [1, 0, 0],
  [0, 0, 1],
  [0, 1, 1]
]
w_value = [
  [0, 2, 0],
  [0, 3, 0],
  [1, 0, 3],
  [1, 1, 0]
]
w_key = torch.tensor(w_key, dtype=torch.float64)
w_query = torch.tensor(w_query, dtype=torch.float64)
w_value = torch.tensor(w_value, dtype=torch.float64)

In [56]:
querys = a @ w_query
keys = a @ w_key
values = b @ w_value

print(querys)

print(keys)

print(values)

tensor([[ 6.,  5., 11.],
        [ 3.,  9., 13.],
        [ 9.,  1., 11.]], dtype=torch.float64)
tensor([[ 9., 13.,  2.],
        [11., 14.,  1.],
        [ 6., 12.,  4.]], dtype=torch.float64)
tensor([[19., 41., 27.],
        [ 9., 21., 12.],
        [12., 17.,  9.]], dtype=torch.float64)


In [53]:
attn_scores = querys @ keys.T
attn_scores

tensor([[141., 147., 140.],
        [170., 172., 178.],
        [116., 124., 110.]], dtype=torch.float64)

In [54]:
attn_scores_softmax = F.softmax(attn_scores, dim=-1)
attn_scores_softmax = attn_scores_softmax / torch.sqrt(torch.tensor(3))
attn_scores_softmax

tensor([[1.4263e-03, 5.7540e-01, 5.2470e-04],
        [1.9314e-04, 1.4271e-03, 5.7573e-01],
        [1.9361e-04, 5.7716e-01, 4.7992e-07]], dtype=torch.float64)

In [57]:
attn_scores_softmax @ values

tensor([[ 5.2120, 12.1508,  6.9480],
        [ 6.9253,  9.8253,  5.2039],
        [ 5.1981, 12.1282,  6.9311]], dtype=torch.float64)